# LSTM Deep Learning Predictions

## import dependancies

In [1]:
import os
import sys

import pandas as pd
#pd.set_option('display.max_rows', None)
# pd.options.display.float_format = '{:, .2f}'.format
pd.set_option('display.max_colwidth',500)
pd.set_option('display.max_columns', 100)

import numpy as np
from numpy import save, load
from numpy import savez_compressed
from scipy.sparse import csr_matrix
from scipy.sparse import vstack
import copy
import pickle

#from scipy.misc import comb, logsumexp
from sklearn.manifold import TSNE #a tool to visualize high dimensional data
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD # dimensionality reduction using truncated SVD (AKA LSA)
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn import preprocessing

import nltk
from nltk.tokenize import word_tokenize
from nltk import FreqDist
from nltk.corpus import stopwords
from nltk.corpus import gutenberg
from nltk.collocations import *
import string #python module
import re # python regex module
from nltk.tokenize import RegexpTokenizer
from nltk.stem import SnowballStemmer
from nltk.tokenize import sent_tokenize

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

np.random.seed(0)

from sklearn.preprocessing import normalize
from functools import reduce

import tensorflow as tf
import keras
from keras.models import Sequential, Model
from keras.layers import Input, Embedding, Conv1D, GlobalMaxPooling1D, Dense, Flatten, MaxPooling1D, Activation, Dropout, LSTM,GlobalMaxPool1D 
from keras.callbacks import ModelCheckpoint
from keras import optimizers
from keras import backend
# from keras import layers
# from keras import models
from keras.preprocessing import sequence

In [3]:
# load pre-processed data

from numpy import load
target_y = load('model_target_data.npz')
target_y = target_y['arr_0']
target_y = np.ravel(target_y)
print(target_y.shape)

features_x =  load('model_data.npz')
features_x = features_x['arr_0']
print(features_x.shape)

(75385,)
(75385, 836)


In [6]:
# split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features_x, labels, test_size = 0.25, random_state = 1)
print(X_train.shape)
print(X_train.ndim)
print(y_train.ndim)

(56538, 836)
2
2


In [7]:
# instantiate a Sequential keras model and add layers
lstm_model = Sequential()
lstm_model.add(Embedding(20000, 128))
lstm_model.add(LSTM(50, return_sequences=True))
lstm_model.add(GlobalMaxPool1D())
lstm_model.add(Dropout(0.5))
lstm_model.add(Dense(50, activation='relu'))
lstm_model.add(Dropout(0.5))
lstm_model.add(Dense(4, activation='softmax'))

In [8]:
lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
lstm_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 128)         2560000   
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 50)          35800     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 50)                0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 50)                2550      
_________________________________________________________________
dropout_2 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 4)                

In [13]:
%%time
history = lstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test), verbose = 1)

Train on 56538 samples, validate on 18847 samples
Epoch 1/10
56538/56538 [==============================] - 1308s 23ms/step - loss: 0.2721 - accuracy: 0.8882 - val_loss: 0.2718 - val_accuracy: 0.8880
Epoch 2/10
56538/56538 [==============================] - 1314s 23ms/step - loss: 0.2640 - accuracy: 0.8915 - val_loss: 0.2685 - val_accuracy: 0.8937
Epoch 3/10
56538/56538 [==============================] - 1310s 23ms/step - loss: 0.2544 - accuracy: 0.8935 - val_loss: 0.2880 - val_accuracy: 0.8915
Epoch 4/10
56538/56538 [==============================] - 1308s 23ms/step - loss: 0.2478 - accuracy: 0.8974 - val_loss: 0.2565 - val_accuracy: 0.8952
Epoch 5/10
56538/56538 [==============================] - 1312s 23ms/step - loss: 0.2400 - accuracy: 0.9000 - val_loss: 0.2652 - val_accuracy: 0.8966
Epoch 6/10
56538/56538 [==============================] - 1315s 23ms/step - loss: 0.2340 - accuracy: 0.9023 - val_loss: 0.2599 - val_accuracy: 0.8997
Epoch 7/10
56538/56538 [==========================

In [ ]:
from keras.callbacks import ModelCheckpoint
file_path = "lstm_model.h5"
checkpoint = ModelCheckpoint(file_path, monitor = 'loss', verbose = 1, save_best_only = True, mode = 'min')


In [ ]:
MAX_SEQUENCE_LENGTH = 1000
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

In [ ]:
model = Sequential()
model.add(Conv1D(32, 5, activation = 'relu', input_shape = (32,32)))
model.add(MaxPooling1D(pool_size=5))
model.add(Conv1D(32, 5, activation = 'relu'))
model.add(MaxPooling1D(pool_size = 5))
model.add(Conv1D(32,5, activation = 'relu'))
model.add(MaxPooling1D(pool_size = 5))
model.add(Flatten())
model.add(Dense(32, activation = 'relu'))
model.add(Dense(4, activation = 'sigmoid'))

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
print(model.summary())

In [ ]:
model.fit(x=X_train, y=y_train, batch_size = 32, epochs = 1, validation_data=(X_test, y_test), verbose = 1)

In [ ]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,))
embedded_sequences = embedding_layer(sequence_input)


# try transpose

In [ ]:
import copy
x_train_transposed = copy.deepcopy(X_train)
x_test_transposed = copy.deepcopy(X_test)

y_train_T = copy.deepcopy(y_train)
y_test_T = copy.deepcopy(y_test)

In [ ]:
x_train_transposed = x_train_transposed.T
x_test_transposed = x_test_transposed.T

y_train_T = y_train_T.T
y_test_T = y_test_T.T 

In [ ]:
x_train_transposed.shape

In [ ]:
#def define_model():
model_T = Sequential()
model_T.add(Conv1D(64,3, activation = 'relu', input_shape = (56538, 1)))
model_T.add(MaxPooling1D(pool_size = 3, strides = 2))
model_T.add(Conv1D(32, 3, activation = 'relu'))
model_T.add(MaxPooling1D(pool_size = 3, strides = 2))
model_T.add(Flatten())
model_T.add(Dense(100, activation = 'relu'))
model_T.add(Dropout(0.3))
model_T.add(Dense(4, activation = 'sigmoid'))

model_T.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
print(model_T.summary())

In [ ]:
model_T.fit(x=x_train_transposed, y=y_train_T, batch_size = 32, epochs = 1, validation_data=(x_test_transposed, y_test_T))